# Code

In [ ]:
from __future__ import print_function
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils

In [ ]:
cudnn.benchmark = True

manualSeed = random.randint(1, 10000)
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

Random Seed:  4802


In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.Resize(64),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ColorJitter(brightness = .5, hue = .3),
    transforms.RandomPerspective(distortion_scale = 0.2, p = 1.0),
    transforms.RandomRotation(degrees=(0, 180)),
    transforms.RandomApply([transforms.RandomSolarize(threshold = 0.75)], 0.5),
])

dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

cat_indices = [i for i in range(len(dataset)) if dataset[i][1] == 3] 
catset = torch.utils.data.Subset(dataset, cat_indices)

# non_cat_indices = [i for i in range(len(dataset)) if i not in cat_indices]
# non_cat_set = torch.utils.data.Subset(dataset, non_cat_indices)

# final_trainset = torch.utils.data.ConcatDataset([catset, non_cat_set])
# print(len(final_trainset))

Files already downloaded and verified


In [ ]:
nc=3

dataloader = torch.utils.data.DataLoader(catset, batch_size=128, shuffle=True, num_workers=2)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

ngpu = 1
nz = 100
ngf = 64
ndf = 64

In [ ]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

In [ ]:
class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d(nz, ngf * 8, kernel_size=2, stride=2, padding=0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, kernel_size=2, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d(ngf * 4, ngf * 2, kernel_size=2, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d(ngf * 2, ngf, kernel_size=2, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. (ngf) x 32 x 32
            nn.ConvTranspose2d(ngf, nc, kernel_size=2, stride=2, padding=1, bias=False),
            nn.Tanh()
            # state size. (nc) x 64 x 64
        )

    def forward(self, input):
        if input.is_cuda and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.main, input, range(self.ngpu))
        else:
            output = self.main(input)
            return output

In [ ]:
netG = Generator(ngpu).to(device)
netG.apply(weights_init)
#load weights to test the model
#netG.load_state_dict(torch.load('weights/netG_epoch_24.pth'))
print(netG)

Generator(
  (main): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(2, 2), stride=(2, 2), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(512, 256, kernel_size=(2, 2), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(256, 128, kernel_size=(2, 2), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(128, 64, kernel_size=(2, 2), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(64, 3, kernel_size=(2, 2), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()
  )
)


In [ ]:
class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is (nc) x 64 x 64
            nn.Conv2d(nc, ndf, kernel_size=2, stride=2, padding=1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, kernel_size=2, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, kernel_size=2, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, kernel_size=2, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 4 x 4
            nn.Conv2d(ndf * 8, 1, kernel_size=2, stride=1, padding=0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        if input.is_cuda and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.main, input, range(self.ngpu))
        else:
            output = self.main(input)

        return output.view(-1, 1).squeeze(1)

In [ ]:
netD = Discriminator(ngpu).to(device)
netD.apply(weights_init)
#load weights to test the model 
#netD.load_state_dict(torch.load('weights/netD_epoch_24.pth'))
print(netD)

Discriminator(
  (main): Sequential(
    (0): Conv2d(3, 64, kernel_size=(2, 2), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(64, 128, kernel_size=(2, 2), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(128, 256, kernel_size=(2, 2), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(256, 512, kernel_size=(2, 2), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): Conv2d(512, 1, kernel_size=(2, 2), stride=(1, 1), bias=False)
    (12): Sigmoid()
  )
)


In [ ]:
criterion = nn.BCEWithLogitsLoss()

optimizerD = optim.Adam(netD.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=0.0002, betas=(0.5, 0.999))

fixed_noise = torch.randn(128, nz, 1, 1, device=device)
real_label = 1
fake_label = 0

niter = 5
g_loss = []
d_loss = []

In [ ]:
netD.train()
netG.train()

for epoch in range(niter):
    for i, data in enumerate(dataloader, 0):
        # Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        # train with real
        netD.zero_grad()
        real_cpu = data[0].to(device)
        print(data)
        batch_size = real_cpu.size(0)
        if batch_size!=128:
          continue

        label = torch.full((batch_size,), real_label, dtype=torch.float, device=device)
        
        output = netD(real_cpu).view(-1)
        output = output.reshape(128, -1)
        # output = output.mean(dim=1, keepdim=True)
        # print(output.shape)  # torch.Size([128, 1])
        
        errD_real = criterion(output[:, 0], label)
        errD_real.backward()
        D_x = output.mean().item()

        # train with fake
        noise = torch.randn(batch_size, nz, 1, 1, device=device)
        fake = netG(noise)
        label.fill_(fake_label)
        output = netD(fake.detach())
        errD_fake = criterion(output, label)
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        errD = errD_real + errD_fake
        optimizerD.step()

        # (2) Update G network: maximize log(D(G(z)))
        netG.zero_grad()
        label.fill_(real_label)  # fake labels are real for generator cost
        output = netD(fake)
        errG = criterion(output, label)
        errG.backward()
        D_G_z2 = output.mean().item()
        optimizerG.step()

        #print('\n[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f / %.4f' % (epoch+1, niter, i+1, len(dataloader), errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))
        
        #save the output
        if i % 100 == 0:
            print('\nSaving the output!')
            vutils.save_image(real_cpu,'real_samples.png',normalize=True)
            fake = netG(fixed_noise)
            vutils.save_image(fake.detach(),'fake_samples_epoch_%03d.png' % (epoch), normalize=True)
    
    # Check pointing for every epoch
    torch.save(netG.state_dict(), 'netG_epoch_%d.pth' % (epoch))
    torch.save(netD.state_dict(), 'netD_epoch_%d.pth' % (epoch))

Streaming output truncated to the last 5000 lines.
          [ 0.8588,  0.8510,  0.8353,  ..., -0.0431, -0.0431, -0.0431],
          [ 0.8588,  0.8510,  0.8353,  ..., -0.0902, -0.1059, -0.1137],
          ...,
          [-0.3333, -0.3569, -0.3961,  ..., -0.3098, -0.2157, -0.1686],
          [-0.4510, -0.4431, -0.4353,  ..., -0.2863, -0.1922, -0.1451],
          [-0.5059, -0.4902, -0.4588,  ..., -0.2784, -0.1843, -0.1373]]]]), tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3])]
[tensor([[[[-0.9294, -0.9373, -0.9608,  ...,  0.1373,  0.1216,  0.1137],
          [-0.9373, -0.9451, -0.9608,  ...,  0.1451,  0.1294,  

KeyboardInterrupt: ignored

# Code 2

In [ ]:
# Necessary Imports

import torch
import torchvision
from torchvision import transforms
import random
import numpy as np

# Define Handler Class to create Non - Synthetic Image Dataset

class NonSyntheticDataset(torchvision.datasets.CIFAR10):

    def __init__(self, root, train = True, transform = None, 
                 target_transform = None, download = False):
        super().__init__(root, train = train, transform = transform,
                         target_transform = target_transform, 
                         download = download)

        if train:
            self.data, self.targets = self.create_imbalance(self.data,
                                                            self.targets)
            
            self.data, self.targets = self.generate_data(self.data,
                                                         self.targets)

    
    def create_imbalance(self, data, targets):

        """
        Remove 99% of the cat images from the dataset. Each category has
        5000 images. Therefore, remove 4950 cat images.
        """

         # Cat index is 3 in CIFAR - 10

        target_indices = [i for i, target in enumerate(targets) if target == 3] 
        target_indices = random.sample(target_indices, k = 4950)

        new_data = np.array([img for i, img in enumerate(data) if i not in target_indices])
        new_targets = [label for i, label in enumerate(targets) if i not in target_indices]

        return new_data, new_targets

    
    def generate_data(self, data, targets):

        """
        Use the remaining 50 cat images to create 4950 new cat images and add 
        them to the dataset.
        """
        new_data = []
        new_targets = []

        req_indices = [i for i, target in enumerate(targets) if target == 3]
        target_data = np.array([img for i, img in enumerate(data) if i in req_indices])

        while len(new_data) != 4950:

            target_image = target_data[random.randint(0, len(target_data) - 1)]
            target_image = self.add_augmentation(target_image)
            
            new_data.append(target_image)
            new_targets.append(3)  # Cat index is 3

        new_data = np.array(new_data)

        return np.concatenate((data, new_data)), (targets + new_targets)

    
    def add_augmentation(self, data):

        """
        Create a new augmented image using the input image.
        """

        # Define the transformations to be applied

        transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.ColorJitter(brightness = .5, hue = .3),
            transforms.RandomPerspective(distortion_scale = 0.2, p = 1.0),
            transforms.RandomRotation(degrees=(0, 180)),
            transforms.RandomApply([transforms.RandomSolarize(threshold = 0.75)], 0.5),
        ])

        # Apply the transformations to the image

        augmented_img = np.array(transform(data))
        
        return augmented_img

In [ ]:
SEED = 42
BATCH_SIZE = 128
EPOCHS = 50

CIFAR_10_MEANS = (0.4914, 0.4822, 0.4465)
CIFAR_10_STDS = (0.2023, 0.1994, 0.2010)

PATH = "/content/models"
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
# Global Constants

SEED = 42
BATCH_SIZE = 128
EPOCHS = 50

CIFAR_10_MEANS = (0.4914, 0.4822, 0.4465)
CIFAR_10_STDS = (0.2023, 0.1994, 0.2010)

PATH = "/content/models"
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
# Setting Seeds for Reproducibility

random.seed(SEED)
np.random.seed(SEED)

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)


torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
train_transform = transforms.Compose([transforms.RandomCrop(32,
                                                            padding = 4),
                                      transforms.RandomHorizontalFlip(), 
                                      transforms.ToTensor(), 
                          transforms.Normalize(CIFAR_10_MEANS,
                                               CIFAR_10_STDS)])

test_transform = transforms.Compose([transforms.ToTensor(),
                                     transforms.Normalize(CIFAR_10_MEANS,
                                                          CIFAR_10_STDS)])

trainset = NonSyntheticDataset(root = './data', train = True, 
                               transform = train_transform,
                               download = True)

trainloader = torch.utils.data.DataLoader(trainset,
                                          batch_size = BATCH_SIZE, 
                                          shuffle = True, 
                                          num_workers = 2, 
                                    worker_init_fn = np.random.seed(SEED))

testset = torchvision.datasets.CIFAR10(root = './data', train = False, 
                                       download = True, 
                                       transform = test_transform)

testloader = torch.utils.data.DataLoader(testset, 
                                         batch_size = BATCH_SIZE,
                                         shuffle = False, 
                                         num_workers = 2, 
                                        worker_init_fn = np.random.seed(SEED))

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
fixed_noise = torch.randn(128, nz, 1, 1, device=device)
netD.train()
netG.train()

for epoch in range(niter):
    print("here")
    for i, data in enumerate(trainloader, 0):
        # Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        # train with real
        netD.zero_grad()
        real_cpu = data[0].to(device)
        batch_size = real_cpu.size(0)
        if batch_size!=128:
          continue

        label = torch.full((batch_size,), real_label, dtype=torch.float, device=device)
        output = netD(real_cpu).view(-1)
        output = output.reshape(128, -1)
        # output = output.mean(dim=1, keepdim=True)
        # print(output.shape)  # torch.Size([128, 1])
        
        errD_real = criterion(output[:, 0], label)
        errD_real.backward()
        D_x = output.mean().item()

        # train with fake
        noise = torch.randn(batch_size, nz, 1, 1, device=device)
        fake = netG(noise)
        label.fill_(fake_label)
        output = netD(fake.detach())
        errD_fake = criterion(output, label)
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        errD = errD_real + errD_fake
        optimizerD.step()

        # (2) Update G network: maximize log(D(G(z)))
        netG.zero_grad()
        label.fill_(real_label)  # fake labels are real for generator cost
        output = netD(fake)
        errG = criterion(output, label)
        errG.backward()
        D_G_z2 = output.mean().item()
        optimizerG.step()

        # print('\n[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f D(x): %.4f' % (epoch+1, niter, (i%40)+1, len(dataloader), errD.item(), errG.item(), D_x))
        
        #save the output
        if i % 100 == 0:
            print('\nSaving the output!')
            vutils.save_image(real_cpu,'real_samples.png',normalize=True)
            fake = netG(fixed_noise)
            vutils.save_image(fake.detach(),'fake_samples_epoch_%03d.png' % (epoch), normalize=True)
    
    # Check pointing for every epoch
    torch.save(netG.state_dict(), 'netG_epoch_%d.pth' % (epoch))
    torch.save(netD.state_dict(), 'netD_epoch_%d.pth' % (epoch))

here

Saving the output!

Saving the output!

Saving the output!

Saving the output!
here

Saving the output!

Saving the output!

Saving the output!

Saving the output!
here

Saving the output!

Saving the output!

Saving the output!

Saving the output!
here

Saving the output!

Saving the output!

Saving the output!

Saving the output!
here

Saving the output!

Saving the output!

Saving the output!

Saving the output!
